## 1. Execute full pipeline

In [1]:
import random
import time

from keras_tuner import RandomSearch

import numpy as np

import pandas as pd

import tensorflow as tf

from config.constants import (
    FORECAST_HORIZON, NB_TRIALS,
    OBSERVATION_WINDOW, SEED, TRAIN_PERC
)

from src.cut_point_detector import CutPointMethod, CutPointModel, get_cut_point_detector
from src.dataset import read_dataset, split_X_y, split_train_test
from src.forecaster import InternalForecaster, TimeSeriesHyperModel
from src.scaler import Scaler
from src.utils import get_error_results

tf.get_logger().setLevel('ERROR')

np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)


In [2]:
print("Available devices:", tf.config.list_physical_devices())
print("Is GPU available?", tf.config.list_physical_devices('GPU'))


Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Is GPU available? [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
if tf.config.list_physical_devices("GPU"):
    try:
        tf.config.set_visible_devices([], "GPU")  # Hide traditional GPUs if any (like external ones)
        tf.config.experimental.set_memory_growth(tf.config.list_physical_devices("GPU")[0], True)
        print("✅ TensorFlow is using Apple GPU via MPS")
    except Exception as e:
        print("⚠️ Error setting MPS GPU:", e)
else:
    print("❌ No GPU found, running on CPU")

✅ TensorFlow is using Apple GPU via MPS


In [4]:
timestamp = 'validate_pipeline'
dataset_domain_argv = 'UCI'
dataset_argv = 'APPLIANCES_ENERGY'
cut_point_model_argv = 'Window'
cut_point_method_argv = 'Cosine'

In [5]:
execution_id = f"{timestamp}_{dataset_domain_argv}_{dataset_argv}_{cut_point_model_argv}_{cut_point_method_argv}_{SEED}"

In [6]:
print(f"Extracting cut point model enum ({cut_point_model_argv})")
cut_point_model = CutPointModel.from_str(cut_point_model_argv)

print(f"Extracting cut point model enum ({cut_point_method_argv})")
cut_point_method = CutPointMethod.from_str(cut_point_method_argv)

print(f"Reading dataset {dataset_argv} from {dataset_domain_argv}")
df, variables = read_dataset(dataset_domain_argv, dataset_argv)
print(f"Variables: {variables}")

Extracting cut point model enum (Window)
Extracting cut point model enum (Cosine)
Reading dataset APPLIANCES_ENERGY from UCI
Variables: ['T_out', 'Press_mm_hg', 'RH_out', 'Windspeed', 'Visibility', 'Tdewpoint']


In [7]:
print("Splitting data into train and test")
train, test = split_train_test(df)

print("Initializing report")
cut_point_approach = f"{cut_point_model.value.title()} {cut_point_method.value.title()}"
report = {
    'execution_id': execution_id,
    'timestamp': timestamp,
    'cut_point_model': cut_point_model.value,
    'cut_point_method': cut_point_method.value,
    'cut_point_approach': cut_point_approach,
    'seed': SEED,
    'observation_window': OBSERVATION_WINDOW,
    'train_perc': TRAIN_PERC,
    'nb_trials': NB_TRIALS,
    'dataset_domain': dataset_domain_argv,
    'dataset': dataset_argv,
    'variables': variables,
    'dataset_shape': df.shape,
    'train_shape': train.shape,
    'test_shape': test.shape,
}
report

Splitting data into train and test
Initializing report


{'execution_id': 'validate_pipeline_UCI_APPLIANCES_ENERGY_Window_Cosine_42',
 'timestamp': 'validate_pipeline',
 'cut_point_model': 'Window',
 'cut_point_method': 'Cosine',
 'cut_point_approach': 'Window Cosine',
 'seed': 42,
 'observation_window': 14,
 'train_perc': 0.8,
 'nb_trials': 15,
 'dataset_domain': 'UCI',
 'dataset': 'APPLIANCES_ENERGY',
 'variables': ['T_out',
  'Press_mm_hg',
  'RH_out',
  'Windspeed',
  'Visibility',
  'Tdewpoint'],
 'dataset_shape': (19735, 7),
 'train_shape': (15788, 7),
 'test_shape': (3947, 7)}

In [8]:
print(f"Started cut point for {cut_point_approach}")
start_time = time.time()
cut_point_detector = get_cut_point_detector(cut_point_model, cut_point_method)
cut_point, cut_point_perc = cut_point_detector.find_cut_point(train, variables)
end_time = time.time()
cut_duration = end_time - start_time
print(f"Cut point: {cut_point}, Cut point percentage: {cut_point_perc}")
print(f"Finished cut point for {cut_point_approach}, duration: {cut_duration}")

report.update({
    'cut_duration': cut_duration,
    'cut_point': str(cut_point),
    'cut_point_perc': cut_point_perc
})
report

Started cut point for Window Cosine
Cut point: 9520, Cut point percentage: 60.29896123638206
Finished cut point for Window Cosine, duration: 1.1111152172088623


{'execution_id': 'validate_pipeline_UCI_APPLIANCES_ENERGY_Window_Cosine_42',
 'timestamp': 'validate_pipeline',
 'cut_point_model': 'Window',
 'cut_point_method': 'Cosine',
 'cut_point_approach': 'Window Cosine',
 'seed': 42,
 'observation_window': 14,
 'train_perc': 0.8,
 'nb_trials': 15,
 'dataset_domain': 'UCI',
 'dataset': 'APPLIANCES_ENERGY',
 'variables': ['T_out',
  'Press_mm_hg',
  'RH_out',
  'Windspeed',
  'Visibility',
  'Tdewpoint'],
 'dataset_shape': (19735, 7),
 'train_shape': (15788, 7),
 'test_shape': (3947, 7),
 'cut_duration': 1.1111152172088623,
 'cut_point': '9520',
 'cut_point_perc': 60.29896123638206}

In [9]:
print("Applying subset to train based on cut point")
reduced_train = cut_point_detector.apply_cut_point(train, cut_point)

print("Training and applying scaler")
scaler = Scaler(variables)
scaled_reduced_train = scaler.fit_scale(reduced_train)
scaled_test = scaler.scale(test)

Applying subset to train based on cut point
Training and applying scaler


In [10]:
print("Splitting into X and y")
X_reduced_scaled_train, y_reduced_scaled_train = split_X_y(scaled_reduced_train)
X_scaled_test, y_scaled_test = split_X_y(scaled_test)

Splitting into X and y


In [11]:
y_reduced_scaled_train[0]

array([[-1.2769759 ,  1.80887286,  0.33542457, -0.8491712 ,  2.30064099,
        -1.43301107],
       [-1.31775067,  1.80574547,  0.3841325 , -0.8491712 ,  2.28501363,
        -1.43987921],
       [-1.35852544,  1.80261809,  0.43284043, -0.8491712 ,  2.26938626,
        -1.44634334],
       [-1.39930022,  1.79949069,  0.48154835, -0.8491712 ,  2.2537589 ,
        -1.45321148],
       [-1.44007499,  1.79636329,  0.53025628, -0.8491712 ,  2.23813153,
        -1.46007961],
       [-1.48084976,  1.79323591,  0.57896421, -0.8491712 ,  2.22250416,
        -1.46654374],
       [-1.52162453,  1.79010852,  0.62767214, -0.8491712 ,  2.2068768 ,
        -1.47341188]])

In [12]:
y_reduced_scaled_train[1]

array([[-1.31775067,  1.80574547,  0.3841325 , -0.8491712 ,  2.28501363,
        -1.43987921],
       [-1.35852544,  1.80261809,  0.43284043, -0.8491712 ,  2.26938626,
        -1.44634334],
       [-1.39930022,  1.79949069,  0.48154835, -0.8491712 ,  2.2537589 ,
        -1.45321148],
       [-1.44007499,  1.79636329,  0.53025628, -0.8491712 ,  2.23813153,
        -1.46007961],
       [-1.48084976,  1.79323591,  0.57896421, -0.8491712 ,  2.22250416,
        -1.46654374],
       [-1.52162453,  1.79010852,  0.62767214, -0.8491712 ,  2.2068768 ,
        -1.47341188],
       [-1.54065276,  1.78072634,  0.63984912, -0.92685418,  2.2068768 ,
        -1.49361228]])

In [13]:
y_reduced_scaled_train[2]

array([[-1.35852544,  1.80261809,  0.43284043, -0.8491712 ,  2.26938626,
        -1.44634334],
       [-1.39930022,  1.79949069,  0.48154835, -0.8491712 ,  2.2537589 ,
        -1.45321148],
       [-1.44007499,  1.79636329,  0.53025628, -0.8491712 ,  2.23813153,
        -1.46007961],
       [-1.48084976,  1.79323591,  0.57896421, -0.8491712 ,  2.22250416,
        -1.46654374],
       [-1.52162453,  1.79010852,  0.62767214, -0.8491712 ,  2.2068768 ,
        -1.47341188],
       [-1.54065276,  1.78072634,  0.63984912, -0.92685418,  2.2068768 ,
        -1.49361228],
       [-1.55696267,  1.77134417,  0.6520261 , -1.00453716,  2.2068768 ,
        -1.51381269]])

In [14]:
print(f"Started running HPO and NAS for {cut_point_approach}")
forecaster_hypermodel = TimeSeriesHyperModel(
    n_variables=len(variables)
)
forecaster_tuner = RandomSearch(
    forecaster_hypermodel,
    objective='val_loss',
    max_trials=3,
    executions_per_trial=1,
    directory=f"outputs/tuner/{execution_id}",
    project_name=f"{cut_point_model.value}_{cut_point_method.value}",
    seed=SEED,
    overwrite=True,
        distribution_strategy=tf.distribute.MirroredStrategy()
)

Started running HPO and NAS for Window Cosine


In [15]:
print("NaN in X_train:", np.isnan(X_reduced_scaled_train).sum())
print("Inf in X_train:", np.isinf(X_reduced_scaled_train).sum())
print("NaN in y_train:", np.isnan(y_reduced_scaled_train).sum())
print("Inf in y_train:", np.isinf(y_reduced_scaled_train).sum())

NaN in X_train: 0
Inf in X_train: 0
NaN in y_train: 0
Inf in y_train: 0


In [16]:
start_time = time.time()
forecaster_tuner.search(
    X_reduced_scaled_train,
    y_reduced_scaled_train,
    validation_split=(1 - TRAIN_PERC),
    shuffle=False,
)
end_time = time.time()
tuner_duration = end_time - start_time

Trial 3 Complete [00h 02m 17s]
val_loss: 0.5396261937916279

Best val_loss So Far: 0.4067087861998328
Total elapsed time: 00h 03m 45s


In [17]:
best_trial = forecaster_tuner.oracle.get_best_trials(num_trials=1)[0]
best_forecaster_model = forecaster_tuner.get_best_models(num_models=1)[0]
print(f"Finished running HPO and NAS for {cut_point_approach}, duration: {tuner_duration}")

print(f"Trial ID: {best_trial.trial_id}")
print(f"Hyperparameters: {best_trial.hyperparameters.values}")
print(f"Score: {best_trial.score}")
print("-" * 40)

Finished running HPO and NAS for Window Cosine, duration: 225.31953024864197
Trial ID: 1
Hyperparameters: {'num_layers': 2, 'units_0': 128, 'learning_rate': 0.005, 'units_1': 96, 'units_2': 32, 'units_3': 64, 'batch_size': 96, 'epochs': 150}
Score: 0.4067087861998328
----------------------------------------


/Users/zamith/Doutorado/TSForecastingSubsets/env/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 26 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [18]:
print("Retrieving best model")
best_forecaster_model.summary()
best_forecaster_model = InternalForecaster(
    best_forecaster_model,
    len(variables),
    best_trial.hyperparameters.values['batch_size'],
    best_trial.hyperparameters.values['epochs'],
)

Retrieving best model


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 14, 128)        │        69,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 14, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 96)             │        86,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 96)             │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 42)             │         4,074 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 6)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 160,490 (626.91 KB)

 Trainable params: 160,042 (625.16 KB)

 Non-trainable params: 448 (1.75 KB)

In [19]:
print("Retraining best model")
start_time = time.time()
best_forecaster_model.fit(
    X_reduced_scaled_train,
    y_reduced_scaled_train,
    shuffle=False
)
end_time = time.time()
retrain_duration = end_time - start_time

Retraining best model
Epoch 1/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - loss: 0.7886 - learning_rate: 0.0050
Epoch 2/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.8015 - learning_rate: 0.0050
Epoch 3/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - loss: 0.7321 - learning_rate: 0.0050
Epoch 4/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.6947 - learning_rate: 0.0050
Epoch 5/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - loss: 0.6970 - learning_rate: 0.0050
Epoch 6/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.7177 - learning_rate: 0.0050
Epoch 7/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.7197 - learning_rate: 0.0050
Epoch 8/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.7431 - learning_rate: 0.0050
Epoch 9/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - loss: 0.7402 - learning_rate: 0.0050
Epoch 10/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - loss: 0.7437 - learning_rate: 0.0050
Epoch 11/150
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 0.7293

In [20]:
print("Running forecasting")
y_scaled_pred = best_forecaster_model.forecast(X_scaled_test)
y_scaled_test_flat = y_scaled_test.reshape(-1, len(variables))
y_scaled_pred_flat = y_scaled_pred.reshape(-1, len(variables))

Running forecasting


2025-02-26 13:20:07.681293: W tensorflow/core/framework/dataset.cc:959] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


111/123 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step

2025-02-26 13:20:08.467387: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]


123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [21]:
y_scaled_test_flat

array([[-0.10809911,  0.77683384,  0.09188493, -1.3152691 ,  0.23782867,
         0.04969852],
       [-0.06732434,  0.78621601,  0.03100002, -1.3152691 ,  0.23782867,
         0.0537386 ],
       [-0.02654957,  0.79559819, -0.02988489, -1.3152691 ,  0.23782867,
         0.06181876],
       ...,
       [ 3.91501168,  0.0637887 , -1.61289257, -0.07234135, -1.13737955,
         3.65749053],
       [ 3.86064532,  0.0637887 , -1.5885386 ,  0.00534163, -1.05924272,
         3.61708973],
       [ 3.83346214,  0.0637887 , -1.56418464,  0.08302462, -0.98110589,
         3.61708973]])

In [22]:
y_scaled_pred_flat

array([[-0.12966685,  0.35573637,  0.11157167, -1.0735902 ,  0.19815555,
         0.04628195],
       [-0.11390005,  0.36686575,  0.11834162, -1.0546783 ,  0.13072836,
         0.03066215],
       [-0.1096265 ,  0.3719384 ,  0.08415096, -1.0322793 ,  0.05227466,
         0.00737556],
       ...,
       [ 1.685428  , -0.12996204, -1.5869994 ,  0.13921669, -0.18103527,
         0.59951776],
       [ 1.6715355 , -0.09873385, -1.5845326 ,  0.15618095, -0.29308334,
         0.5845803 ],
       [ 1.6121542 , -0.04202348, -1.5635628 ,  0.21599913, -0.12213793,
         0.5153472 ]], dtype=float32)

In [23]:
print("Calculating error")
y_test = scaler.descale(pd.DataFrame(y_scaled_test_flat, columns=variables))
y_pred = scaler.descale(pd.DataFrame(y_scaled_pred_flat, columns=variables))

Calculating error


In [24]:
y_test

,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,7.70,759.000000,79.666667,1.000000,40.000000,4.37
1,7.85,759.050000,78.833333,1.000000,40.000000,4.38
2,8.00,759.100000,78.000000,1.000000,40.000000,4.40
3,8.12,759.116667,77.333333,1.000000,37.833333,4.38
4,8.23,759.133333,76.666667,1.000000,35.666667,4.37
...,...,...,...,...,...,...
27484,22.70,755.200000,55.666667,3.333333,23.666667,13.30
27485,22.60,755.200000,56.000000,3.500000,24.500000,13.30
27486,22.50,755.200000,56.333333,3.666667,25.333333,13.30
27487,22.30,755.200000,56.666667,3.833333,26.166667,13.20


In [25]:
y_pred

,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint
0,7.620658,756.755859,79.936119,1.518515,39.576885,4.361543
1,7.678660,756.815186,80.028778,1.559090,38.857769,4.322881
2,7.694381,756.842224,79.560814,1.607147,38.021057,4.265242
3,7.732622,756.826355,79.159996,1.657856,37.565918,4.206795
4,7.689744,756.798340,78.386337,1.813423,36.718426,4.153632
...,...,...,...,...,...,...
27484,14.265692,754.490234,58.884140,3.694121,36.313931,6.115121
27485,14.177341,754.487549,57.422970,3.930127,35.726940,5.839324
27486,14.297930,754.167480,56.687733,4.120559,35.532795,5.730912
27487,14.246822,754.333862,56.721497,4.156955,34.337795,5.693938


In [26]:
len(X_scaled_test)

3927

In [27]:
len(X_scaled_test) * FORECAST_HORIZON

27489

In [28]:
error_results = get_error_results(y_test, y_pred, variables)
print(f"Obtained error results: {error_results}")

Obtained error results: {'Avg_MAPE': 2384777619229.833, 'Avg_MAE': 3.7413104994549626, 'Avg_MSE': 30.19562449877661, 'Avg_RMSE': 4.897442951439135, 'Avg_R2': 0.4201735114783411, 'Avg_WAPE': 0.025251256641381885, 'T_out_MAPE': 0.18587107692225582, 'T_out_MAE': 2.9037142264777507, 'T_out_MSE': 14.892385647882566, 'T_out_RMSE': 3.859065385282111, 'T_out_R2': 0.39439354003382054, 'T_out_WAPE': 0.21253742493648559, 'Press_mm_hg_MAPE': 0.005203438628917185, 'Press_mm_hg_MAE': 3.940274692373199, 'Press_mm_hg_MSE': 22.271627791248914, 'Press_mm_hg_RMSE': 4.719282550478295, 'Press_mm_hg_R2': 0.29488667578182526, 'Press_mm_hg_WAPE': 0.005212042890934138, 'RH_out_MAPE': 0.12253450118347443, 'RH_out_MAE': 7.070294026750273, 'RH_out_MSE': 79.95632190512016, 'RH_out_RMSE': 8.941829896901426, 'RH_out_R2': 0.775648517807439, 'RH_out_WAPE': 0.09906937776327776, 'Windspeed_MAPE': 2229156031445.242, 'Windspeed_MAE': 1.0179763072268253, 'Windspeed_MSE': 1.635318801685478, 'Windspeed_RMSE': 1.2787958405021

In [29]:
print("Writing report")
report.update({
    'tuner_duration': tuner_duration,
    'retrain_duration': retrain_duration,
    'total_duration': cut_duration + tuner_duration + retrain_duration,
    'error_results': error_results,
    'scaled_reduced_train_shape': scaled_reduced_train.shape,
    'best_trial_id': best_trial.trial_id,
    'best_trial_hyperparameters': best_trial.hyperparameters.values,
    'best_trial_score': best_trial.score,
    'best_forecaster_model': best_forecaster_model.summary(),
})

Writing report


In [30]:
report

{'execution_id': 'validate_pipeline_UCI_APPLIANCES_ENERGY_Window_Cosine_42',
 'timestamp': 'validate_pipeline',
 'cut_point_model': 'Window',
 'cut_point_method': 'Cosine',
 'cut_point_approach': 'Window Cosine',
 'seed': 42,
 'observation_window': 14,
 'train_perc': 0.8,
 'nb_trials': 15,
 'dataset_domain': 'UCI',
 'dataset': 'APPLIANCES_ENERGY',
 'variables': ['T_out',
  'Press_mm_hg',
  'RH_out',
  'Windspeed',
  'Visibility',
  'Tdewpoint'],
 'dataset_shape': (19735, 7),
 'train_shape': (15788, 7),
 'test_shape': (3947, 7),
 'cut_duration': 1.1111152172088623,
 'cut_point': '9520',
 'cut_point_perc': 60.29896123638206,
 'tuner_duration': 225.31953024864197,
 'retrain_duration': 65.24015593528748,
 'total_duration': 291.6708014011383,
 'error_results': {'Avg_MAPE': 2384777619229.833,
  'Avg_MAE': 3.7413104994549626,
  'Avg_MSE': 30.19562449877661,
  'Avg_RMSE': 4.897442951439135,
  'Avg_R2': 0.4201735114783411,
  'Avg_WAPE': 0.025251256641381885,
  'T_out_MAPE': 0.18587107692225582,

## 2. What would be the error if we predicted the average values for all variables (Dummy Forecaster)?

In [31]:
X_train, y_train = split_X_y(train)
X_test, y_test = split_X_y(test)

In [32]:
train_targets_flat = pd.DataFrame(y_train.reshape(-1, len(variables)), columns=variables)
avg_values = train_targets_flat.mean(axis=0).to_numpy()

In [33]:
n_test = y_test.shape[0]
dummy_pred = np.tile(avg_values, (n_test, FORECAST_HORIZON, 1))

dummy_pred_flat = dummy_pred.reshape(-1, len(variables))
y_test_flat = pd.DataFrame(y_test.reshape(-1, len(variables)), columns=variables)

In [34]:
dummy_error_results = get_error_results(y_test_flat, dummy_pred_flat, variables)
print(f"Error metrics for Dummy Forecaster (predicting average values): \n{dummy_error_results}")

Error metrics for Dummy Forecaster (predicting average values): 
{'Avg_MAPE': 3425021718969.223, 'Avg_MAE': 7.215418809609981, 'Avg_MSE': 118.70230356109563, 'Avg_RMSE': 9.023975290726012, 'Avg_R2': -0.8137139190732777, 'Avg_WAPE': 0.04869908342626568, 'T_out_MAPE': 0.5581505737110357, 'T_out_MAE': 8.04392388105092, 'T_out_MSE': 85.53065629469543, 'T_out_RMSE': 9.24827855845051, 'T_out_R2': -2.4781477730923025, 'T_out_WAPE': 0.5887751805856862, 'Press_mm_hg_MAPE': 0.00621866023244673, 'Press_mm_hg_MAE': 4.703571354779968, 'Press_mm_hg_MSE': 31.910844092516054, 'Press_mm_hg_RMSE': 5.648968409587369, 'Press_mm_hg_R2': -0.01028813733691325, 'Press_mm_hg_WAPE': 0.006221702179580035, 'RH_out_MAPE': 0.32693204736892706, 'RH_out_MAE': 17.103435976028628, 'RH_out_MSE': 465.69433115411005, 'RH_out_RMSE': 21.579952065611963, 'RH_out_R2': -0.30670359708489037, 'RH_out_WAPE': 0.23965435572387742, 'Windspeed_MAPE': 4903590906267.885, 'Windspeed_MAE': 1.742090696100894, 'Windspeed_MSE': 4.1720749928

In [35]:
df_comparison = pd.DataFrame({
    "Trained Model": pd.Series(error_results),
    "Dummy Forecaster": pd.Series(dummy_error_results)
})

df_comparison = df_comparison.round(5)
df_comparison

,Trained Model,Dummy Forecaster
Avg_MAPE,2.384778e+12,3.425022e+12
Avg_MAE,3.741310e+00,7.215420e+00
Avg_MSE,3.019562e+01,1.187023e+02
Avg_RMSE,4.897440e+00,9.023980e+00
Avg_R2,4.201700e-01,-8.137100e-01
Avg_WAPE,2.525000e-02,4.870000e-02
T_out_MAPE,1.858700e-01,5.581500e-01
T_out_MAE,2.903710e+00,8.043920e+00
T_out_MSE,1.489239e+01,8.553066e+01
T_out_RMSE,3.859070e+00,9.248280e+00
